# Import Packages

In [147]:
import pandas as pd
import requests
import io
import json
import time
from oauthlib.oauth2 import LegacyApplicationClient
from requests_oauthlib import OAuth2Session
import regex as re

# Location Entries - Envoy API

In [ ]:
# Apply Login info
oauth = OAuth2Session(client = LegacyApplicationClient(client_id = client_id))
token = oauth.fetch_token(token_url = token_url,
                          username = username, 
                          password = password,
                          client_id = client_id,
                          client_secret = client_secret,
                          scope = scope)

In [ ]:
# Create start and end dates to prevent pulling every piece of data
today = date.today()
yesterday = today - timedelta(days = 1)

# Define the URL
entries_url = f'https://app.envoy.com/a/visitors/api/v2/entries/?start_date={start_date}&end_date={end_date}'

# Call the API to get the final offset number
entries = oauth.get(entries_url).json()['meta']['total']

# Empty Dataframe to Append to
envoy_entries = pd.DataFrame()


'''Paginate to get all data, convert from JSON to Dataframe, flatten the JSON columns'''
# Start off by defining the offset at 0 
offset = 0

# While offset is < final_offset, run this loop
while offset < final_offset:
    
    # Take the offset from above and tag it onto the end of this url, converting the data to a JSON
    entries = oauth.get(f'{entries_url}&page[limit]=20&page[offset]={str(offset)}').json()

    # Convert to JSON
    entries_json = json.loads(entries.text)
    
    # Convert to pandas dataframe
    entries_df = pd.json_normalize(entries_json['data'])

    # Append dataframe to Master DF
    envoy_entries = envoy_entries.append(entries_df)
    
    # Add 20 to the offset value
    offset += 20

# Location Data - Envoy API

In [148]:
'''Grab the Locations information from the API'''
# Note this is the same as above, just changes from the entries table to the locations table
locations = oauth.get('https://app.envoy.com/a/visitors/api/v2/locations/')

# Convert to JSON
locations_json = json.loads(locations.text)

# Convert to pandas dataframe
locations_df = pd.json_normalize(locations_json['data'])

# Reverse Geocoding - USPS API

In [149]:
# Define the headers
headers = {'Authorization': 
           'Bearer API_KEY',
          }

# Create Empty list to append to 
fips_list = []

# Loop through each zip code in the zip code column
for zip_code in locations_df['Zip Code']:
    
    # Call the api, passing the zip code as an argument
    response = requests.get(f'https://www.huduser.gov/hudapi/public/usps?type=2&query={zip_code}', headers=headers).json()
    
    # Append the geoid to the FIPS list
    fips_list.append(response['data']['results'][0]['geoid'])
    
    # Sleep for 2 seconds
    time.sleep(2)
    
# Create a dataframe from the two lists
fips_zips = pd.DataFrame({'Zip Code': zip_codes,'FIPS': fips_list} )

In [150]:
# Join back with Office Zip Codes
us_offices = locations_df.merge(fips_zips, left_on = 'Zip Code', right_on = 'Zip Code')

# Drop the duplicate zip code column
us_offices = us_offices.drop(['Zip Code'], axis=1)

# Join office back to Entries
envoy_entries = envoy_entries.merge(us_offices, left_on = 'Office ID', right_on = 'Office ID')

# Drop the duplicate zip code column
envoy_entries = envoy_entries.drop(['Office ID'], axis=1)

# Covid Data - CovidActNow API

In [108]:
# Enter API Key to reference later
api_key = 'API_KEY'

# Generate empty dataframe
covid_act_now = pd.DataFrame()

# Initiate the for loop
for fip in us_offices['FIPS']: 
    
    # Call the API with requests
    response = requests.get(f'https://api.covidactnow.org/v2/county/{fip}.json?apiKey={api_key}')

    # Convert to JSON
    covid_json = json.loads(response.text)

    # Convert to Dataframe
    covid_df = pd.json_normalize(covid_json)
    
    # Append to dataframe
    covid_act_now = covid_act_now.append(covid_df)

In [ ]:
# Join office back to Entries
envoy_entries = envoy_entries.merge(covid_act_now, left_on = 'Office ID', right_on = 'Office ID')

# Drop the duplicate zip code column
envoy_entries = envoy_entries.drop(['Office ID'], axis=1)